In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns', None)
import datetime
import catboost
from catboost import CatBoostClassifier,Pool
from time import time
from tqdm import tqdm_notebook as tqdm
import os
import random
import itertools
import json
import pprint
import gc
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,GroupKFold
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import cohen_kappa_score

In [ ]:
def qwk(a1, a2, max_rat=3):
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e


In [ ]:
def spec(value,*args):
    i= specs[specs['event_id'] == value].index.values[-1]
    print('Index :',i)
    print('Event_code :',train[train['event_id'] == value]['event_code'].unique()[-1])
    for arg in args:
        if(arg == 'info'):
         print(specs[arg][i])
        elif(arg == 'args'):
         print(pprint.pprint(json.loads(specs[arg][i])))
        else:
         print('Nothing')

In [ ]:
def event(value):
    i = train[train['event_id'] == value].index.values[-1]
    print(pprint.pprint(json.loads(train['event_data'][i])))

In [ ]:
def get_game_feature(df,data):
    g0_d = dict(df[df['type'] == 'Game'].groupby('installation_id')['type'].value_counts())
    g0 = {}
    for (k,i),v in g0_d.items():
        g0[k] = v
    del g0_d
    df['G0'] = df['installation_id'].map(g0)
    df['G0'].fillna(0,inplace=True)

    g1_d = dict(df[(df['event_code'] == 4020) & (df['type'] == 'Game')].groupby('installation_id')['event_code'].value_counts())
    g1 = {}
    for (k,i),v in g1_d.items():
        g1[k] = v
    del g1_d
    df['G1'] = df['installation_id'].map(g1)
    df['G1'].fillna(0,inplace=True)
    del g1
    
    df['contains_true'] = df[(df['event_code'] == 4020) & (df['type'] == 'Game')]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')>= 0) else 0)
    g2 = dict(df[df['event_code'] == 4020].groupby('installation_id')['contains_true'].sum())
    df['G2'] = df['installation_id'].map(g2)
    df['G2'].fillna(0,inplace=True)
    for c in ['contains_true']:
        df.pop(c)
        
    df['G3'] = df['G2'] / df['G1']
    df['G3'] = df['G3'].map(lambda x:3 if(x == 1.0) else(2 if((x >= 0.5) & (x <1.0)) else (1 if((x>0.0) &  (x<0.5)) else 0)))
    df['G3'].fillna(0,inplace=True)
    
    i = 4
    for d in tqdm(data):
        g_attempt_d = dict(df[(df['type'] == 'Game') & (df['event_code'] == 4020) & (df['title'] == d)].groupby('installation_id')['title'].value_counts())
        g_attempt = {}
        for (k,z),v in g_attempt_d.items():
            g_attempt[k] = v

        df[f'G{i}_attempt'] = df['installation_id'].map(g_attempt)
        df[f'G{i}_attempt'].fillna(0,inplace=True)

        df['contains_true'] = df[(df['type'] == 'Game') & (df['event_code'] == 4020) & (df['title'] == d)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')>=0) else 0)
        g = dict(df.groupby('installation_id')['contains_true'].sum())
        df[f'G{i}_correct'] = df['installation_id'].map(g)
        df[f'G{i}_correct'].fillna(0,inplace=True)

        df[f'G{i}_accuracy'] = df[f'G{i}_correct'] / df[f'G{i}_attempt']
        df[f'G{i}_accuracy'] = df[f'G{i}_accuracy'].map(lambda x:3 if(x == 1.0) else(2 if((x >= 0.5) & (x <1.0)) else (1 if((x>0.0) &  (x<0.5)) else 0)))
        df[f'G{i}_accuracy'].fillna(0,inplace=True)

        for c in ['contains_true',f'G{i}_attempt']:
            df.pop(c)
            
        i = i + 1

In [ ]:
def get_attempted_assessment_results(df,data):
    if(data == 'train'):
        df = df[(df['type'] == 'Assessment') & (((df['event_code'] == 4100) & (df['title'] != 'Bird Measurer (Assessment)')) | ((df['event_code'] == 4110) & (df['title'] == 'Bird Measurer (Assessment)')))]
    else:
        df =  df[(df['type'] == 'Assessment') &  ((df['event_count'] == 1) | ((df['event_code'] == 4100) & (df['title'] != 'Bird Measurer (Assessment)')) | ((df['event_code'] == 4110) & (df['title'] == 'Bird Measurer (Assessment)')))]
    session_count = df['game_session'].value_counts().to_dict()
    df['assessment_attempt_count'] = df['game_session'].map(session_count)

    df['contains_true_assessment'] = df['event_data'].map(lambda x: True if (x.find('"correct":true')>=0) else False)

    change_value = {
        True : 1,
        False : 0
    }
    df['contains_true_assessment'] = df['contains_true_assessment'].map(change_value)

    correct_attempt = dict(df.groupby('game_session',sort=False)['contains_true_assessment'].sum())
    df['contains_true_assessment_count'] = df['game_session'].map(correct_attempt)

    for c in ['contains_true_assessment']:
        df.pop(c)

    df['accumulated_accuracy'] = np.where((df['contains_true_assessment_count'] == 0),0,(df['contains_true_assessment_count']/df['assessment_attempt_count']))

    df.loc[(df['type'] == 'Assessment'), 'accuracy_group'] = 0
    df.loc[(df['accumulated_accuracy'] == 1) & (df['type'] == 'Assessment'), 'accuracy_group'] = 3
    df.loc[(df['accumulated_accuracy'] == 0.5) & (df['type'] == 'Assessment'), 'accuracy_group'] = 2
    df.loc[(df['accumulated_accuracy'] < 0.5) & (df['accumulated_accuracy'] > 0) & (df['assessment_attempt_count'] > 0) & (df['type'] == 'Assessment'), 'accuracy_group'] = 1

    df.rename(columns = {'contains_true_assessment_count': 'num_correct',
                            'accumulated_accuracy':'accuracy',
                            'assessment_attempt_count': 'total_attempt'},inplace=True)
    df = df.drop_duplicates(subset = 'game_session',keep = 'last')

In [ ]:
def get_validation_data(df,num_samples,random_state):
    validation_data = df[(df['type'] == 'Assessment') & (((df['event_code'] == 4100) & (df['title'] != 'Bird Measurer (Assessment)')) | ((df['event_code'] == 4110) & (df['title'] == 'Bird Measurer (Assessment)')))]
    validation_data.drop_duplicates(subset = 'game_session',keep = 'last',inplace=True)
    if(isinstance(num_samples,float)):
            validation_data = validation_data.sample(frac = num_samples,random_state = random_state)
            print(validation_data.shape)
    else:
            validation_data = validation_data.sample(n = num_samples,random_state = random_state)
            print(validation_data.shape)
    return validation_data

In [ ]:
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')

In [ ]:
train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')

In [ ]:
assessment_id = list(train[train['type'] == 'Assessment']['installation_id'].unique())
train = train.loc[train['installation_id'].isin(assessment_id)]

In [ ]:
in_train_game = train[(train['type'] == 'Assessment') & (((train['event_code'] == 4100) & (train['title'] != 'Bird Measurer (Assessment)')) | ((train['event_code'] == 4110) & (train['title'] == 'Bird Measurer (Assessment)')))]['game_session'].unique().tolist()
train_game_session = train['game_session'].unique().tolist()
not_in_train_game = [x for x in train_game_session if x not in in_train_game]
print(len(in_train_game),len(train_game_session),len(not_in_train_game))

train = train.loc[~train['game_session'].isin(not_in_train_game)]
print(train.shape)

In [ ]:
train = train.reset_index(drop=False)
train.drop(columns = ['index'],axis = 1, inplace = True)

In [ ]:
list_of_event_code = set(train['event_code'].unique()).union(set(test['event_code'].unique()))
list_of_event_code = list(list_of_event_code)

In [ ]:
a = train.copy()
b = a[a['type'] == 'Assessment']
b = b.drop_duplicates(subset = 'installation_id',keep ='last').index.tolist()
a = a.drop_duplicates(subset = 'installation_id', keep = 'last').index.tolist()

In [ ]:
train_game = train[(train['type'] == 'Game') | (train['type'] == 'Assessment')]
train_activity = train[(train['type'] == 'Assessment') | (train['type'] == 'Activity')]
train_clip = train[(train['type'] == 'Assessment') | (train['type'] == 'Clip')]

In [ ]:
def get_features(df:pd.DataFrame(),which):
    
    new_df_list_values = []
    event_code_count_dict = {eve : 0 for eve in list_of_event_code}
    new_df_dict = {}
        
    df = df.reset_index(drop= False)
    df.drop(columns=['index'],axis=1,inplace=True)
    
    for i,data in df.groupby('installation_id',sort=False):
        new_data = data[df.columns.tolist()]
        a1 = new_data[new_data['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'first').index.tolist()
        a2 = new_data[new_data['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'last').index.tolist()

        a = []
        a.append(0)
        print(len(a1),len(a2))
        for i in range(len(a2)):
            a.append(a2[i])
            a.append(a2[i]+1)
        #a.pop(len(a)-1)

        for i in tqdm(range(0,len(a),2)):
            new_df = data.iloc[a[i]:a[i+1],:]
            g_session = new_df[new_df['type'] == 'Assessment']['game_session'].unique()[-1]
            print(i,g_session)
            new_df_dict[g_session] = {}
            new_df_dict[g_session]['num_game_session'] = new_df['game_session'].nunique()
            new_df_dict[g_session]['num_event_id'] = new_df['event_id'].nunique()
            new_df_dict[g_session]['num_actions_before_assessment'] = len(new_df['event_code'])
            new_df_dict[g_session][f'num_unique_{which}'] = new_df[new_df['type'] == which]['title'].nunique()
            if(which == 'Game'):
                new_df_dict[g_session]['game_attempts'] = new_df[new_df['event_code'] == 4020]['event_code'].count()
                new_df_dict[g_session]['mean_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].mean()
                new_df_dict[g_session]['max_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].max()
                new_df_dict[g_session]['min_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].min()
                new_df_dict[g_session]['std_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].std()
                new_df['gaming_accuracy'] = new_df[(new_df['type'] == 'Game') & (new_df['event_code'] == 4020)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')) else 0)
                new_df['gaming_accuracy'].fillna(0,inplace=True)
                new_df_dict[g_session]['gaming_accuracy'] = new_df['gaming_accuracy'].sum()/new_df_dict[g_session]['game_attempts']
            elif(which == 'Activity'):
                new_df_dict[g_session]['Activity actions'] = new_df[new_df['type'] == 'Activity']['event_id'].count()

            for k,v in event_code_count_dict.items():
                new_df_dict[g_session][k] = v

            for i in event_code_count_dict.keys():
                 new_df_dict[g_session][i] = new_df[new_df['event_code'] == i]['event_code'].count()
            #break
    return new_df_dict
        
        

In [ ]:
g = get_features(train_game,'Game')
g

In [ ]:
train_labels.head(11)

In [ ]:
for i , data in train.groupby('installation_id',sort = False):
    new_data = data[train.columns.tolist()]
    a1 = new_data[new_data['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'first').index.tolist()
    a2 = new_data[new_data['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'last').index.tolist()
    
    a = []
    a.append(0)
    for i in range(len(a2)):
        a.append(a2[i])
        a.append(a2[i]+1)
    a.pop(len(a)-1)
    break
a

**For Game**

In [ ]:
#game_list = set(train[train['type'] == 'Game']['title'].unique()).union(set(test[test['type'] == 'Game']['title'].unique()))
#game_list = list(game_list)zzz

In [ ]:
#get_game_feature(train,game_list)

In [ ]:
#get_game_feature(test,game_list)

In [ ]:
#activities_list = set(train[train['type'] == 'Activity']['title'].unique()).union(set(test[test['type'] == 'Activity']['title'].unique()))
#activities_list = list(activities_list)

In [ ]:
#assessment_list = set(train[train['type'] == 'Assessment']['title'].unique()).union(set(test[test['type'] == 'Assessment']['title'].unique()))
#assessment_list = list(assessment_list)

In [ ]:
#train.shape

**Train**

In [ ]:
train = train[(train['type'] == 'Assessment') & (((train['event_code'] == 4100) & (train['title'] != 'Bird Measurer (Assessment)')) | ((train['event_code'] == 4110) & (train['title'] == 'Bird Measurer (Assessment)')))]

session_count = train['game_session'].value_counts().to_dict()
train['assessment_attempt_count'] = train['game_session'].map(session_count)

train['contains_true_assessment'] = train['event_data'].map(lambda x: True if (x.find('"correct":true')>=0) else False)

change_value = {
    True : 1,
    False : 0
}
train['contains_true_assessment'] = train['contains_true_assessment'].map(change_value)

correct_attempt = dict(train.groupby('game_session',sort=False)['contains_true_assessment'].sum())
train['contains_true_assessment_count'] = train['game_session'].map(correct_attempt)

for c in ['contains_true_assessment']:
    train.pop(c)
    
train['accumulated_accuracy'] = np.where((train['contains_true_assessment_count'] == 0),0,(train['contains_true_assessment_count']/train['assessment_attempt_count']))

train.loc[(train['type'] == 'Assessment'), 'accuracy_group'] = 0
train.loc[(train['accumulated_accuracy'] == 1) & (train['type'] == 'Assessment'), 'accuracy_group'] = 3
train.loc[(train['accumulated_accuracy'] == 0.5) & (train['type'] == 'Assessment'), 'accuracy_group'] = 2
train.loc[(train['accumulated_accuracy'] < 0.5) & (train['accumulated_accuracy'] > 0) & (train['assessment_attempt_count'] > 0) & (train['type'] == 'Assessment'), 'accuracy_group'] = 1

train.rename(columns = {'contains_true_assessment_count': 'num_correct',
                        'accumulated_accuracy':'accuracy',
                        'assessment_attempt_count': 'total_attempt'},inplace=True)
train = train.drop_duplicates(subset = 'game_session',keep = 'last')
train = train.reset_index(drop=False)
train.drop(columns = ['index'],axis = 1, inplace = True)
train.shape

**Test**

In [ ]:
test =  test[(test['type'] == 'Assessment') &  ((test['event_count'] == 1) | ((test['event_code'] == 4100) & (test['title'] != 'Bird Measurer (Assessment)')) | ((test['event_code'] == 4110) & (test['title'] == 'Bird Measurer (Assessment)')))]

In [ ]:
session_count = test['game_session'].value_counts().to_dict()
test['assessment_attempt_count'] = test['game_session'].map(session_count)

test['contains_true_assessment'] = test['event_data'].map(lambda x: True if (x.find('"correct":true')>=0) else False)

change_value = {
    True : 1,
    False : 0
}
test['contains_true_assessment'] = test['contains_true_assessment'].map(change_value)

correct_attempt = dict(test.groupby('game_session',sort=False)['contains_true_assessment'].sum())
test['contains_true_assessment_count'] = test['game_session'].map(correct_attempt)


for c in ['contains_true_assessment']:
    test.pop(c)

test['accumulated_accuracy'] = np.where((test['contains_true_assessment_count'] == 0),0,(test['contains_true_assessment_count']/test['assessment_attempt_count']))

test.loc[(test['type'] == 'Assessment'), 'accuracy_group'] = 0
test.loc[(test['accumulated_accuracy'] == 1) & (test['type'] == 'Assessment'), 'accuracy_group'] = 3
test.loc[(test['accumulated_accuracy'] == 0.5) & (test['type'] == 'Assessment'), 'accuracy_group'] = 2
test.loc[(test['accumulated_accuracy'] < 0.5) & (test['accumulated_accuracy'] > 0) & (test['assessment_attempt_count'] > 0) & (test['type'] == 'Assessment'), 'accuracy_group'] = 1

test.rename(columns = {'contains_true_assessment_count': 'num_correct',
                        'accumulated_accuracy':'accuracy',
                        'assessment_attempt_count': 'total_attempt'},inplace=True)

test = test.drop_duplicates(subset = 'game_session',keep = 'last')
test = test.reset_index(drop=False)
test.drop(columns = ['index'],axis = 1, inplace = True)
test.shape

In [ ]:
#validation_data = get_validation_data(test,0.2,45)#
#validation_data = validation_data.reset_index(drop=False)
#validation_data.drop(columns = ['index'],axis = 1, inplace = True)

In [ ]:
#validation_install_id = validation_data['installation_id'].unique().tolist()#

In [ ]:
#test = test.loc[~test['installation_id'].isin(validation_install_id)]#
#test.shape

In [ ]:
for c in tqdm(['event_id','game_session','installation_id','event_code','event_data','event_count','type','timestamp','world']):
    for df in [train,test]:
        df.pop(c)

In [ ]:
list_of_user_activities = list(set(train['title'].value_counts().index).union(set(test['title'].value_counts().index)))
activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))

train['title'] = train['title'].map(activities_map)
test['title'] = test['title'].map(activities_map)
train_labels['title'] = train_labels['title'].map(activities_map)

In [ ]:
#validation_data['title'] = validation_data['title'].map(activities_map)

In [ ]:
x = train.drop(columns=['accuracy_group'],axis=1)
y = train['accuracy_group']

In [ ]:
#val_x = validation_data.drop(columns=['accuracy_group'],axis=1)
#val_y = validation_data['accuracy_group']

In [ ]:
def make_classifier():
    clf = CatBoostClassifier(
                               loss_function='MultiClass',
                               task_type="CPU",
                               learning_rate=0.01,
                               iterations=100,
                               od_type="Iter",
                               early_stopping_rounds=50,
                               random_seed=2019,
                               colsample_bylevel=0.87,
                               eval_metric='Kappa',
                              )
        
    return clf
oof = np.zeros(len(x))

In [ ]:
#train.head()

In [ ]:
#y.head(10000)

In [ ]:
oof = np.zeros(len(x))
NFOLDS = 5
folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=2019)


for fold, (trn_idx, test_idx) in enumerate(folds.split(x, y)):
    
    print(f'Training on fold {fold+1}')
    clf = make_classifier()
    clf.fit(x.loc[trn_idx], y.loc[trn_idx], eval_set=(x.loc[test_idx], y.loc[test_idx]),
                          use_best_model=True, verbose=500)
    
    oof[test_idx] = clf.predict(x.loc[test_idx]).reshape(len(test_idx))
    print('OOF QWK:', qwk(y, oof))
    
print('-' * 30)
print('OOF QWK:', qwk(y, oof))
print('-' * 30)

In [ ]:
train[train['installation_id'] == '0006a69f']['world'].unique()

In [ ]:
train[(train['installation_id'] == '0006a69f') & (train['type'] == 'Assessment')]['title'].unique()

In [ ]:
train[(train['world'] == 'MAGMAPEAK') & (train['type'] == 'Game')]['title'].unique()

In [ ]:
train[(train['world'] == 'MAGMAPEAK') & (train['type'] == 'Activity')]['title'].unique()

In [ ]:
train[(train['world'] == 'MAGMAPEAK') & (train['type'] == 'Assessment')]['title'].unique()

In [ ]:
cols = train.columns.tolist()
cols.remove('installation_id')
for i ,data in train.groupby('installation_id',sort=False):
    new_data = data[cols]
    new_data['installation_id'] = i
    break
new_data.head()

In [ ]:
def get_features(df:pd.DataFrame(),which):
    
    new_df_list_values = []
    event_code_count_dict = {}
    new_df_dict = {}
    
    for i in train['event_code'].unique().tolist():
        event_code_count_dict[i] = 0
        
    df = df.reset_index(drop= False)
    df.drop(columns=['index'],axis=1,inplace=True)
    
    a1 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'first').index.tolist()
    a2 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'last').index.tolist()
    
    a = []
    a.append(0)
    for i in range(len(a2)):
        a.append(a2[i])
        a.append(a2[i]+1)
    a.pop(len(a)-1)
    
    for i in tqdm(range(len(a))):
        new_df = df.iloc[a[i]:a[i+1],:]
        ids  = new_df['installation_id'].unique().tolist()[-1]
        new_df_dict[ids] = {}
        new_df_dict['num_game_session'] = new_df['game_session'].nunique()
        new_df_dict['num_event_id'] = new_df['event_id'].nunique()
        new_df_dict['num_actions_before_assessment'] = len(new_df['event_code'])
        new_df_dict[f'num_unique_{which}'] = new_df[new_df['type'] == which]['title'].nunique()
        if(which == 'Game'):
            new_df_dict['game_attempts'] = new_df[new_df['event_code'] == 4020]['event_code'].count()
            new_df_dict['mean_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].mean()
            new_df_dict['max_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].max()
            new_df_dict['min_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].min()
            new_df_dict['std_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].std()
            new_df['gaming_accuracy'] = new_df[(new_df['type'] == 'Game') & (new_df['event_code'] == 4020)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')) else 0)
            new_df['gaming_accuracy'].fillna(0,inplace=True)
            new_df_dict['gaming_accuracy'] = new_df['gaming_accuracy'].sum()/new_df_dict['game_attempts']
            #new_df['time_gap_before_assessment'] = new_df['']
        elif(which == 'Activity'):
            new_df_dict['Activity actions'] = new_df[new_df['type'] == 'Activity']['event_id'].count()
        
        event_code_count_dict.update(new_df_dict)
        break
    print(event_code_count_dict)

In [ ]:
def get_features(df:pd.DataFrame(),which):
    
    new_df_list_values = []
    event_code_count_dict = {eve : 0 for eve in list_of_event_code}
    new_df_dict = {}
        
    df = df.reset_index(drop= False)
    df.drop(columns=['index'],axis=1,inplace=True)
    
    a1 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'first').index.tolist()
    a2 = df[df['type'] == 'Assessment'].drop_duplicates(subset = 'game_session',keep = 'last').index.tolist()
    
    a = []
    a.append(0)
    for i in range(len(a2)):
        a.append(a2[i])
        a.append(a2[i]+1)
    a.pop(len(a)-1)
    
    for i in tqdm(range(2)):
        new_df = df.iloc[a[i]:a[i+1],:]
        ids  = new_df['installation_id'].unique().tolist()[-1]
        g_session = new_df[new_df['type'] == 'Assessment']['game_session'].unique()[-1]
        print(ids,g_session)
        new_df_dict[ids] = {}
        new_df_dict[ids][g_session] = {}
        new_df_dict[ids][g_session]['num_game_session'] = new_df['game_session'].nunique()
        new_df_dict[ids][g_session]['num_event_id'] = new_df['event_id'].nunique()
        new_df_dict[ids][g_session]['num_actions_before_assessment'] = len(new_df['event_code'])
        new_df_dict[ids][g_session][f'num_unique_{which}'] = new_df[new_df['type'] == which]['title'].nunique()
        if(which == 'Game'):
            new_df_dict[ids][g_session]['game_attempts'] = new_df[new_df['event_code'] == 4020]['event_code'].count()
            new_df_dict[ids][g_session]['mean_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].mean()
            new_df_dict[ids][g_session]['max_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].max()
            new_df_dict[ids][g_session]['min_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].min()
            new_df_dict[ids][g_session]['std_game_time'] = new_df[new_df['type'] == 'Game']['game_time'].std()
            new_df['gaming_accuracy'] = new_df[(new_df['type'] == 'Game') & (new_df['event_code'] == 4020)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')) else 0)
            new_df['gaming_accuracy'].fillna(0,inplace=True)
            new_df_dict[ids][g_session]['gaming_accuracy'] = new_df['gaming_accuracy'].sum()/new_df_dict[ids][g_session]['game_attempts']
        elif(which == 'Activity'):
            new_df_dict[ids][g_session]['Activity actions'] = new_df[new_df['type'] == 'Activity']['event_id'].count()
        
        for k,v in event_code_count_dict.items():
            new_df_dict[ids][g_session][k] = v
        
        for i in event_code_count_dict.keys():
             new_df_dict[ids][g_session][i] = new_df[new_df['event_code'] == i]['event_code'].count()
        #break
    return new_df_dict
        
        